# Location id per numero di ordini

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import copy


In [ ]:
# -----------------------------
# 1. Carica i dati
# -----------------------------
delivery_data = pd.read_csv('delivery_history.csv')

# -----------------------------
# 2. Rendiamo is_event un booleano sicuro
# -----------------------------
# Controlliamo che is_event abbia solo valori convertibili in booleano
valid_bool_mask = delivery_data['is_event'].isin([0, 1, True, False])
invalid_is_event = delivery_data.loc[~valid_bool_mask, 'is_event'].unique()
print("\n❌ Valori non convertibili in booleani in 'is_event':", invalid_is_event)

# Convertiamo solo i valori validi a booleano
delivery_data.loc[valid_bool_mask, 'is_event'] = delivery_data.loc[valid_bool_mask, 'is_event'].astype(bool)

# -----------------------------
# 3. Escludiamo righe dove is_event non è True
# -----------------------------
delivery_data = delivery_data[delivery_data['is_event'] == True].copy()

# -----------------------------
# 4. Filtra coordinate latitudine e longitudine (escludi points fuori range)
# -----------------------------
delivery_data = delivery_data[
    (delivery_data['lon'] >= 5) & (delivery_data['lon'] <= 12.5)
]

# -----------------------------
# 5. Escludiamo sabati e domeniche
# Supponendo che ci sia una colonna datetime 'date' o simile
# Se non c'è, bisognerebbe creare o specificare come avere il campo data
# -----------------------------
# Convertiamo la colonna data in datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# Drop righe con date non valide
delivery_data = delivery_data.dropna(subset=['delivery_date'])

# -----------------------------

# Prendiamo solo i giorni della settimana da lun (0) a ven (4)
delivery_data = delivery_data[delivery_data['delivery_date'].dt.dayofweek < 5]

# -----------------------------
# 6. Filtra solo ottobre,novembre,dicembre

# Filtra solo i mesi ottobre, novembre e dicembre
delivery_data = delivery_data[delivery_data['delivery_date'].dt.month.isin([10, 11, 12])]

# -----------------------------


# -----------------------------
# 6. Elimina duplicati per location_id lasciandone uno solo
# -----------------------------
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates(subset='location_id').reset_index(drop=True)


# Stampa info per verifica
print(f"\nDataset preprocessato: {len(delivery_points)} punti univoci dopo filtri e rimozione weekend filtrando solo per ottobre, novembre e dicembre.")


In [ ]:
# -----------------------------
# 1. Filtra solo ottobre, novembre, dicembre e lun-ven
# -----------------------------
months_to_consider = [10, 11, 12]
filtered_q4 = delivery_data[
    (delivery_data['delivery_date'].dt.month.isin(months_to_consider)) &
    (delivery_data['delivery_date'].dt.dayofweek < 5)  # lun=0,...,ven=4
].copy()

# Ordina per data
filtered_q4 = filtered_q4.sort_values('delivery_date')

# -----------------------------
# 2. Calcola il lunedì della settimana
# -----------------------------
filtered_q4['week_start_date'] = filtered_q4['delivery_date'] - pd.to_timedelta(
    filtered_q4['delivery_date'].dt.weekday, unit='d'
)

# -----------------------------
# 3. Conta ordini settimanali per cliente
# -----------------------------
orders_per_week = (
    filtered_q4.groupby(['week_start_date', 'location_id'])
    .size()
    .reset_index(name='orders_week')
)

max_orders = 5
# Consideriamo solo clienti che hanno da 1 a 5 ordini a settimana
orders_per_week = orders_per_week[orders_per_week['orders_week'].between(1, max_orders)]

# Lista settimane
weeks = sorted(orders_per_week['week_start_date'].unique())

# -----------------------------
# 4. Conta clienti per numero di ordini
# -----------------------------
clients_per_order_count = []
for week in weeks:
    data_week = orders_per_week[orders_per_week['week_start_date'] == week]
    counts = data_week['orders_week'].value_counts()
    counts = counts.reindex(range(1, max_orders + 1), fill_value=0)
    clients_per_order_count.append(counts)

df_clients_per_order = pd.DataFrame(clients_per_order_count, index=weeks).fillna(0).astype(int)

# -----------------------------
# 5. Grafico stacked bar
# -----------------------------
def make_week_label(week_start_date):
    end_date = week_start_date + pd.Timedelta(days=4)  # venerdì
    return f"{week_start_date.strftime('%d-%m')} - {end_date.strftime('%d-%m')}"

weeks_labels_corrected = [make_week_label(w) for w in weeks]

highlight_offset_map = {4: 3, 5: 33}  # offset etichette per ordini 4 e 5

fig, ax = plt.subplots(figsize=(14, 8))
x = np.arange(len(weeks))

bottom = np.zeros(len(weeks))
colors = plt.cm.tab20.colors[1:6]

labels = [f'{i} ordine{"i" if i > 1 else ""}' for i in range(1, max_orders + 1)]

for i, label in enumerate(labels, start=1):
    bars = ax.bar(x, df_clients_per_order[i], bottom=bottom, color=colors[i-1], label=label)
    for bar in bars:
        h = bar.get_height()
        if h > 0:
            label_text = str(int(h))
            if i in highlight_offset_map:
                offset = highlight_offset_map[i]
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + h + offset,
                        label_text, ha='center', va='bottom', fontsize=8, color=colors[i-1])
            else:
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + h/2,
                        label_text, ha='center', va='center', fontsize=8, color='black')
    bottom += df_clients_per_order[i]

ax.set_xticks(x)
ax.set_xticklabels(weeks_labels_corrected, rotation=90)
ax.set_xlabel('Settimana lavorativa (lun-ven)')
ax.set_ylabel('Numero clienti')
ax.set_title('Clienti per numero di ordini settimanali (ottobre-dicembre, lun-ven)')
ax.legend(title='Ordini per settimana')

ax.set_ylim(0, df_clients_per_order.values.max() * 1.4)

plt.tight_layout()
plt.show()


In [ ]:
threshold = 0.85 # 60%

# Dizionario per salvare i clienti nei vari gruppi
client_groups = {
    "sempre 5": [],
    "sempre 4-5": [],
    "sempre 3-5": [],
    "sempre 2-5": [],
    "almeno 1 ordine": [],
    "0 ordini": []
}

# Itera sui clienti
for client, group in orders_per_week.groupby("location_id"):
    total_weeks = len(group)
    if total_weeks == 0:
        client_groups["0 ordini"].append(client)
        continue

    orders_list = group["orders_week"].values

    # Conta frequenze
    freq_5 = np.mean(orders_list == 5)
    freq_45 = np.mean(np.isin(orders_list, [4, 5]))
    freq_545 = np.mean(np.isin(orders_list, [3, 4, 5]))
    freq_2345 = np.mean(np.isin(orders_list, [2, 3, 4, 5]))
    freq_1plus = np.mean(orders_list >= 1)  # almeno 1 ordine

    # Assegna al gruppo più alto soddisfatto
    if freq_5 >= threshold:
        client_groups["sempre 5"].append(client)
    elif freq_45 >= threshold:
        client_groups["sempre 4-5"].append(client)
    elif freq_545 >= threshold:
        client_groups["sempre 3-5"].append(client)
    elif freq_2345 >= threshold:
        client_groups["sempre 2-5"].append(client)
    elif freq_1plus >= threshold:
        client_groups["almeno 1 ordine"].append(client)
    else:
        client_groups["0 ordini"].append(client)

# Stampa risultati
for group_name, clients in client_groups.items():
    print(f"\n{group_name} ({len(clients)} clienti):")
    print(clients)


In [ ]:
import pickle

# Salva in un file pickle
with open("client_groups.pkl", "wb") as f:
    pickle.dump(client_groups, f)

print("✅ Dizionario client_groups salvato in client_groups.pkl")


# tutti non solo OND

In [ ]:
# Carica file
delivery_data = pd.read_csv('delivery_history.csv')

# is_event booleano
valid_bool_mask = delivery_data['is_event'].isin([0, 1, True, False])
invalid_is_event = delivery_data.loc[~valid_bool_mask, 'is_event'].unique()
print("\n❌ Valori non convertibili in 'is_event':", invalid_is_event)
delivery_data.loc[valid_bool_mask, 'is_event'] = delivery_data.loc[valid_bool_mask, 'is_event'].astype(bool)
delivery_data = delivery_data[delivery_data['is_event'] == True].copy()

# Filtra coordinate
delivery_data = delivery_data[
    (delivery_data['lon'] >= 5) & (delivery_data['lon'] <= 12.5)
]

# Data in datetime e drop errori
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')
delivery_data = delivery_data.dropna(subset=['delivery_date'])

# Tieni solo feriali (lunedì-venerdì)
delivery_data = delivery_data[delivery_data['delivery_date'].dt.dayofweek < 5]

# Elimina duplicati
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates(subset='location_id').reset_index(drop=True)

# Info finale
print(f"\nDataset preprocessato: {len(delivery_points)} punti univoci dopo filtri e rimozione weekend (solo feriali).")


In [ ]:
# 1. Aggiungi la colonna 'week_start' che indica il lunedì della settimana per delivery_date
delivery_data['week_start'] = delivery_data['delivery_date'] - pd.to_timedelta(delivery_data['delivery_date'].dt.dayofweek, unit='d')

# 2. Calcola ordini per cliente per settimana
orders_per_week = delivery_data.groupby(['location_id', 'week_start']).size().reset_index(name='orders_week')

# 3. Lista tutte le settimane uniche
settimane = sorted(delivery_data['week_start'].unique())

# 4. Lista clienti unici
clienti = delivery_data['location_id'].unique()

# 5. Crea tutte le combinazioni cliente-settimana (per includere anche settimane senza ordini)
multi_index = pd.MultiIndex.from_product([clienti, settimane], names=['location_id', 'week_start'])
full_df = pd.DataFrame(index=multi_index).reset_index()

# 6. Unisci con orders_per_week, riempi con 0 le settimane senza ordini
full_df = full_df.merge(orders_per_week, how='left', on=['location_id', 'week_start']).fillna(0)

# 7. Assicurati che orders_week sia intero
full_df['orders_week'] = full_df['orders_week'].astype(int)

# full_df è ora pronto per essere usato nella classificazione
print(f"Combinazioni cliente-settimana totali: {len(full_df)}")
print(full_df.head())


In [ ]:
threshold = 0.85

# Calcolo delle frequenze cumulative per cliente
freq_cumulative = full_df.groupby('location_id').apply(
    lambda df: pd.Series({
        '5': (df['orders_week'] == 5).mean(),
        '4_5': df['orders_week'].isin([4,5]).mean(),
        '3_4_5': df['orders_week'].isin([3,4,5]).mean(),
        '2_5_4_5': df['orders_week'].isin([2,3,4,5]).mean(),
        '1_2_5_4_5': df['orders_week'].isin([1,2,3,4,5]).mean(),
    })
)

def assign_category(row):
    if row['5'] >= threshold:
        return '5 volte'
    elif row['4_5'] >= threshold:
        return '4 o 5 volte'
    elif row['3_4_5'] >= threshold:
        return '3, 4 o 5 volte'
    elif row['2_5_4_5'] >= threshold:
        return '2, 3, 4 o 5 volte'
    elif row['1_2_5_4_5'] >= threshold:
        return '1, 2, 3, 4 o 5 volte'
    else:
        return '0 volte'

# Assegna categoria ai clienti
client_categories = freq_cumulative.apply(assign_category, axis=1)

# Salva location_id per categoria in un dizionario
location_ids_per_category = {}
for category in client_categories.unique():
    location_ids_per_category[category] = client_categories[client_categories == category].index.tolist()

# Adesso 'location_ids_per_category' contiene le liste di location_id per ogni categoria
for cat, locs in location_ids_per_category.items():
    print(f"Categoria '{cat}' ha {len(locs)} clienti")

for category, locs in location_ids_per_category.items():
    print(f"\nCategoria '{category}' (totale {len(locs)} clienti):")
    print(locs)  # Stampa primi 10 location_id per categoria
    
    
print(location_ids_per_category)  # Stampa primi 10 location_id per categoria
